<a href="https://colab.research.google.com/github/Mananalik/Food_Freshness_Classification/blob/main/Ensemble_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import joblib
import json
import tensorflow as tf
from google.colab import files

In [ ]:
try:
    model1 = tf.keras.models.load_model('best_model.keras')
    model2 = tf.keras.models.load_model('best_tuned_model.keras')
    scaler = joblib.load('time_series_scaler.pkl')
    with open('freshness_labels.json', 'r') as f:
        freshness_labels = json.load(f)
    print("✅ All models and helpers loaded successfully.")
except Exception as e:
    print(f"❌ Error loading files: {e}. Make sure all files are uploaded to this session.")


✅ All models and helpers loaded successfully.


In [ ]:
print("\nPlease upload the 'live_test_data.csv' file you just created:")
uploaded = files.upload()


Please upload the 'live_test_data.csv' file you just created:


In [ ]:
if uploaded:
    filename = list(uploaded.keys())[0]
    live_df = pd.read_csv(filename)

    if len(live_df) >= 180:
        live_data = live_df[['MQ3', 'MQ8', 'MQ135']].iloc[:180].values


        scaled_data = scaler.transform(live_data)


        scaled_data_batch = np.expand_dims(scaled_data, axis=0)


        probs1 = model1.predict(scaled_data_batch)[0]
        probs2 = model2.predict(scaled_data_batch)[0]


        averaged_probs = (probs1 + probs2) / 2.0


        prediction_index = np.argmax(averaged_probs)
        prediction_label = freshness_labels[str(prediction_index)]
        confidence = averaged_probs[prediction_index]

        print("\n--- FINAL PREDICTION ---")
        print(f" Freshness Stage: {prediction_label}")
        print(f" Confidence: {confidence*100:.2f}%")
    else:
        print(f"❌ Error: The uploaded file has only {len(live_df)} readings. At least 180 are required.")